# LASA recognition

## Sound-alike

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.cluster import KMeans, AffinityPropagation
import matplotlib.pyplot as plt
import nltk
from nltk.metrics.distance import edit_distance
from tqdm.notebook import tqdm
import sys
import pickle
import string
import sys, math, random, copy

In [2]:
df = pd.read_csv("./drugsatfda20211116/Products.txt", sep='\t+', engine='python')
drugNames = df['DrugName']

In [3]:
drugNames = drugNames.drop_duplicates() \
                     .dropna()
random_incides = [np.random.randint(0, len(drugNames)) for _ in range(10)]
drugNames.iloc[random_incides]

6903                          AMRIX
7725                     METAXALONE
43208                        TIVDAK
9623         PENICILLIN G POTASSIUM
1468                           UREX
5584                         PLETAL
18213          ESCITALOPRAM OXALATE
38702                        VYXEOS
22448                        LORYNA
1903     IODOHIPPURATE SODIUM I 131
Name: DrugName, dtype: object

In [128]:
names = np.array(drugNames)
lasa_names = np.unique(np.loadtxt("lists_LASA/sa_ISMP+FDA.txt", dtype=str))
names = np.append(lasa_names, names)

DULoxetine


In [12]:
# Try edit distance between different phonetic forms
# Calculate similarity matrix between letters
# https://www.diva-portal.org/smash/get/diva2:1116701/FULLTEXT01.pdf
neighbors_of = {}
neighbors_of['q'] = ['w', 'c', 'k']
neighbors_of['w'] = ['v', 'u']
neighbors_of['e'] = ['i', 'y', 'a']
neighbors_of['r'] = ['t', 'f', 'd', 'e']
neighbors_of['t'] = ['d', 'f', 'r', 'v', 'p']
neighbors_of['y'] = ['i', 'e', 'a', 'u']
neighbors_of['u'] = ['i', 'y', 'o', 'a', 'e', 'w']
neighbors_of['i'] = ['e', 'y', 'u']
neighbors_of['o'] = ['e', 'u']
neighbors_of['p'] = ['l', 'o', 't']
neighbors_of['a'] = ['e', 'i', 'u', 'y']
neighbors_of['s'] = ['x', 'z', 'c']
neighbors_of['d'] = ['b', 'f', 't', 'p']
neighbors_of['f'] = ['v', 'd', 't']
neighbors_of['g'] = ['j', 'h', 'q']
neighbors_of['h'] = ['f', 'g']
neighbors_of['j'] = ['g', 'c']
neighbors_of['k'] = ['c', 'q']
neighbors_of['l'] = ['m', 'n']
neighbors_of['z'] = ['s', 'x', 'c']
neighbors_of['x'] = ['s', 'c', 'z', 'k']
neighbors_of['c'] = ['k', 's']
neighbors_of['v'] = ['f', 'b', 'c', 'w']
neighbors_of['b'] = ['g', 'n', 'v', 'd']
neighbors_of['n'] = ['m', 'b']
neighbors_of['m'] = ['b', 'n']

keys = sorted(neighbors_of.keys())
dists = {el:{} for el in keys}

# Distance between letters and their neighbours
def distance(start, end, raw):
    if start == end:
        if raw:
            return 0
        else:
            return 1
        
    visited = [start]
    queue = []
    
    for key in neighbors_of[start]:
        queue.append({'char': key, 'dist': 1})
        
    while True:
        key = queue.pop(0)
        visited.append(key['char'])
        if key['char'] == end:
            return key['dist']
        
        for neighbor in neighbors_of[key['char']]:
            if neighbor not in visited:
                queue.append({'char': neighbor, 'dist': key['dist']+1})

In [13]:
# Computes a similarity matrix for letters of the English alphabet
# based on keyboard distances between letters 
def alldists(option, verbose):          
    if option == "raw":
        longest_dist = 0
        avgdist = 0
        for i in range(len(keys)):
            for j in range(len(keys)):
                dists[keys[i]][keys[j]] = distance(keys[i], keys[j], True)
                avgdist += dists[keys[i]][keys[j]]
                if dists[keys[i]][keys[j]] > longest_dist:
                    longest_dist = dists[keys[i]][keys[j]]
        key_dist = longest_dist
        avgdist /= len(keys) ** 2 + 0.0
        
        buckets = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
        for i in range(len(keys)):
            for j in range(len(keys)):
                buckets[dists[keys[i]][keys[j]]] += 1
        if verbose:
            print("Average distance: " + str(avgdist))
            print("Longest distance: " + str(key_dist))
            print("Buckets: " + str(buckets))
            print(str(dists).replace("'", '"'))
    return copy.deepcopy(dists)

In [14]:
# Take all ascii characters
all_ascii = string.printable

# Add the manually computed Edit Distance for letters to the full similarity matrix
# Add hardcoded similarity for the other characters (0 if same character, 12 otherwise)
similarity_dict = alldists("raw", False)
similarity_dict_all = {}

# Construct full similarity matrix by iterating through all ascii characters
for a in all_ascii:
    similarity_dict_all[a] = {}
    for b in all_ascii:
        # If characters are the same, assign 0
        # Otherwise if similarity has alredy been computed, assign that value
        # Otherwise assign 12      
        similarity_dict_all[a][b] = (0 if a == b else similarity_dict[a][b] if a in similarity_dict and b in similarity_dict[a] else 12)
similarity_array = np.zeros((len(similarity_dict), len(similarity_dict)))

for character_index, (character, other_characters) in enumerate(similarity_dict.items()):
    for c_index, c in enumerate(other_characters.values()):
        similarity_array[character_index][c_index] = c

# for i in similarity_dict_all:
#     print(i)
#     print(similarity_dict_all[i])

In [15]:
ins_cost = 3
del_cost = 4

def min_cost_path(cost, operations):    
    # operation at the last cell
    path = [operations[cost.shape[0]-1][cost.shape[1]-1]]    
    # cost at the last cell
    min_cost = cost[cost.shape[0]-1][cost.shape[1]-1]    
    row = cost.shape[0]-1
    col = cost.shape[1]-1    
    while row > 0 and col > 0:            
        if cost[row-1][col-1] <= cost[row-1][col] and cost[row-1][col-1] <= cost[row][col-1]:
            path.append(operations[row-1][col-1])
            row -= 1
            col -= 1
        elif cost[row-1][col] <= cost[row-1][col-1] and cost[row-1][col] <= cost[row][col-1]:
            path.append(operations[row-1][col])
            row -= 1
        else:
            path.append(operations[row][col-1])
            col -= 1                    
    return "".join(path[::-1][1:])

def edit_distance_dp(seq1, seq2):
    # there is no difference between upper and lower case for this application    
    seq1 = seq1.lower()
    seq2 = seq2.lower()
    
    # create an empty 2D matrix to store cost
    cost = np.zeros((len(seq1)+1, len(seq2)+1))
    
    # fill the first row
    cost[0] = [i for i in range(len(seq2)+1)]
    
    # fill the first column
    cost[:, 0] = [i for i in range(len(seq1)+1)]
    
    # to store the operations made
    operations = np.asarray([['-' for j in range(len(seq2)+1)] \
                                 for i in range(len(seq1)+1)])
    
    # fill the first row by insertion 
    operations[0] = ['I' for i in range(len(seq2)+1)]
    
    # fill the first column by insertion operation (D)
    operations[:, 0] = ['D' for i in range(len(seq1)+1)]
    
    operations[0, 0] = '-'
    
    # now, iterate over earch row and column
    for row in range(1, len(seq1)+1):
        
        for col in range(1, len(seq2)+1):
            
            # if both the characters are same then the cost will be same as 
            # the cost of the previous sub-sequence
            if seq1[row-1] == seq2[col-1]:
                cost[row][col] = cost[row-1][col-1]
            else:
                
                insertion_cost = cost[row][col-1] + ins_cost
                deletion_cost = cost[row-1][col] + del_cost
                substitution_cost = cost[row-1][col-1] + similarity_dict_all[seq1[row-1]][seq2[col-1]]
#                 print(f"sim for {seq1[row-1]} and {seq2[col-1]}: {similarity_dict_all[seq1[row-1]][seq2[col-1]]}")
                
                # calculate the minimum cost
                cost[row][col] = min(insertion_cost, deletion_cost, substitution_cost)
                
                # get the operation
                if cost[row][col] == substitution_cost:
                    operations[row][col] = 'S'
                    
                elif cost[row][col] == ins_cost:
                    operations[row][col] = 'I'
                else:
                    operations[row][col] = 'D'
    return cost[len(seq1), len(seq2)], min_cost_path(cost, operations)

edit_distance_dp("novolin", "novolog")

(4.0, '-----SS')

In [16]:
# Levenshtein distance
# n = len(names)
n = 1000
lev_dist = np.zeros((n, n))
lev_sim = np.zeros((n, n))

for i in tqdm(range(n)):
    for j in range(i+1, n):
        ni = names[i]
        nj = names[j]
        # dist = edit_distance(ni, nj)
        dist, operations = edit_distance_dp(ni, nj)
        lev_dist[i, j] = dist
        lev_dist[j, i] = dist

  0%|          | 0/1000 [00:00<?, ?it/s]

In [153]:
def is_row_similar(row, threshold=26):
    sorted_row = sorted(row)[:len(row)//4]
#     lowest = sorted(row)[1]
#     return lowest < threshold
#     return np.average(sorted_row) < threshold
    return True

In [154]:
# Apply thresholding so not all medications get clustered (so not everything gets clasified as LASA)
file_path = 'lev_dist_FDA_newphonetic_1000_wLASA.pickle'
# pickle.dump(lev_dist, open(file_path, "wb"))
lev_dist = pickle.load(open(file_path, "rb"))

filter_lev_dist = []
columns_to_remove = []
for i, row in enumerate(lev_dist):
    if is_row_similar(row):
        filter_lev_dist.append(row)  
    else:
        columns_to_remove.append(i)

# for i, row in enumerate(lev_dist):
#     filter_lev_dist.append(row)  

for i, row in enumerate(filter_lev_dist):
    filter_lev_dist[i] = [entry for c, entry in enumerate(row) if c not in columns_to_remove]
      
filter_lev_dist = np.array(filter_lev_dist)
print(np.shape(filter_lev_dist))
# np.savetxt('filter_lev_sim.txt',filter_lev_sim,fmt='%s')

(1000, 1000)


In [155]:
# Distance to similarity
# Try out other ways to translate distance to similarity
lev_sim = 1 / (1 + filter_lev_dist)
# lev_sim = 1 / (1 + lev_dist)

In [158]:
# Cluster on computed similarities
aff_prop = AffinityPropagation(affinity="precomputed", damping=0.96,max_iter = 1000, verbose=True)
aff_prop.fit(lev_sim)
print(f'Found {len(aff_prop.cluster_centers_indices_)} clusters.')

C:\Users\danie\anaconda3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(


Converged after 229 iterations.
Found 158 clusters.


In [159]:
for cluster_id in range(len(aff_prop.cluster_centers_indices_)):
    exemplar = names[aff_prop.cluster_centers_indices_[cluster_id]]
    members = names[np.nonzero(aff_prop.labels_ == cluster_id)]

    print(f'{cluster_id + 1}. \033[1m{exemplar}\033[0m ({len(members)} members): {", ".join(members)}')


1. DULoxetine (8 members): DULoxetine, FLUoxetine, OLANZapine, PARoxetine, cloZAPine, cycloSERINE, levoFLOXacin, OXYLONE
2. HumuLIN (7 members): HumuLIN, eriBULin, THEELIN, HEDULIN, PHENY-PAS-TEBAMIN, ISMELIN, SULFAMYLON
3. IDArubicin (4 members): DAUNOrubicin, IDArubicin, epiRUBicin, idaruCIZUmab
4. LORazepam (5 members): ALPRAZolam, LORazepam, clonazePAM, diazePAM, levETIRAcetam
5. NovoLIN (9 members): KlonoPIN, LEVOleucovorin, NovoLIN, NovoLOG, metFORMIN, AMINOPHYLLIN, OVULEN, OVULEN-21, OVULEN-28
6. OXcarbazepine (3 members): OXcarbazepine, carBAMazepine, cefTAZidime
7. PAZOPanib (5 members): PAZOPanib, acetaZOLAMIDE, methazolAMIDE, BUTAZOLIDIN, ACYLANID
8. RABEprazole (5 members): ARIPiprazole, PRALAtrexate, RABEprazole, metroNIDAZOLE, NEPTAZANE
9. SUMAtriptan (3 members): SUMAtriptan, ZOLMitriptan, sAXagliptin
10. SUNItinib (7 members): PONATinib, SORAfenib, SUNItinib, tiZANidine, UNITENSEN, CEDILANID-D, METIMYD
11. TEGretol (2 members): TEGretol, TEGRETOL
12. TRENtal (7 members)

In [166]:
clusters = []
all_LASA = []
for cluster_id in range(len(aff_prop.cluster_centers_indices_)):
    exemplar = names[aff_prop.cluster_centers_indices_[cluster_id]]
    member_ind = np.nonzero(aff_prop.labels_ == cluster_id)
    members = names[member_ind]
    most_similar_members = set()
    # For each member (member index) of the cluster, check if it is similar enough to the rest     
    for member in member_ind[0]:
        for datapoint in range(len(filter_lev_dist)):
            # Omit the point itself and it does not have distances below 10, remove it, probably not LASA
            if (member != datapoint and filter_lev_dist[member][datapoint] < 6):
                most_similar_members.add(names[member])
                
    if most_similar_members:
        clusters.append({str(cluster_id) : most_similar_members})
        all_LASA.append(list(most_similar_members))
        print(f'\033[1m{exemplar}\033[0m ({len(most_similar_members)} most similar from {len(members)} total): {", ".join(most_similar_members)}')
# print(clusters)

HumuLIN (3 most similar from 7 total): eriBULin, HumuLIN, HEDULIN
IDArubicin (2 most similar from 4 total): epiRUBicin, IDArubicin
LORazepam (2 most similar from 5 total): LORazepam, clonazePAM
NovoLIN (4 most similar from 9 total): NovoLOG, KlonoPIN, NovoLIN, OVULEN
TEGretol (2 most similar from 2 total): TEGRETOL, TEGretol
TRENtal (4 most similar from 7 total): PRANTAL, TINDAL, SERENTIL, TRENtal
fentaNYL (2 most similar from 3 total): SUFentanil, fentaNYL
quiNINE (2 most similar from 4 total): quiNINE, QUINIDEX
rifAXIMin (2 most similar from 3 total): rifAXIMin, riTUXimab
sulfADIAZINE (2 most similar from 3 total): SULFADIAZINE, sulfADIAZINE
ESTINYL (3 most similar from 6 total): MESTINON, ESTINYL, ACTIDIL
BAL (20 most similar from 79 total): THAM, DISIPAL, LORFAN, EUTONYL, BAL, NARDIL, RAUVAL, FLAXEDIL, VELBAN, EQUANIL, DROLBAN, PLACIDYL, FLAGYL, ELAVIL, TERFONYL, POVAN, TEPANIL, ESIMIL, ISORDIL, PLAQUENIL
ARALEN (4 most similar from 11 total): TALWIN, ARALEN, RITALIN, OXSORALEN
GAN

In [ ]:
# Good results:
# 19. cloZAPine (4 members): QUEtiapine, azaTHIOprine, cloZAPine, clomiPHENE
# OxyCONTIN (3 members): FLUoxetine, OxyCONTIN, oxyMORphone
# 10. oxyCODONE (11 members): QUEtiapine, chlorproMAZINE, clonazePAM, hydrOXYzine, oxyCODONE, AMINOPHYLLIN, FOLVRON, HYPROTIGEN 5%, VI-TWEL, BENEMID, PAMINE
# 53. TENSILON (12 members): DACTINomycin, busPIRone, sulfADIAZINE, STILBESTROL, SUS-PHRINE SULFITE FREE, TENSILON, PROMETHAZINE HYDROCHLORIDE PLAIN, PRO-BANTHINE, SERPANRAY, SERPALAN, RITALIN, METICORTELONE
# 54. TENSILON PRESERVATIVE FREE (6 members): buPROPion, cefTAZidime, VASOXYL, POTASSIUM CHLORIDE, TENSILON PRESERVATIVE FREE, STERANE
# 1. DEPO-Medrol (5 members): DEPO-Medrol, NexAVAR, NovoLOG, PARoxetine, PROzac
# 45. GANTRISIN PEDIATRIC (10 members): HumuLIN, OxyCONTIN, hydrOXYzine, GANTRISIN PEDIATRIC, DIUPRES-250, PLEGINE, PARNATE, TENUATE DOSPAN, VOSOL HC, NOVRAD
# 2. DAPTOmycin (7 members): DAPTOmycin, KlonoPIN, methIMAzole, raNITIdine, riMANTAdine, rifAMPin, ALCOHOL 10% AND DEXTROSE 5%

In [168]:
# Flatten the list for easy LASA check
flat_list = [item for sublist in all_LASA for item in sublist]
print(flat_list)

['eriBULin', 'HumuLIN', 'HEDULIN', 'epiRUBicin', 'IDArubicin', 'LORazepam', 'clonazePAM', 'NovoLOG', 'KlonoPIN', 'NovoLIN', 'OVULEN', 'TEGRETOL', 'TEGretol', 'PRANTAL', 'TINDAL', 'SERENTIL', 'TRENtal', 'SUFentanil', 'fentaNYL', 'quiNINE', 'QUINIDEX', 'rifAXIMin', 'riTUXimab', 'SULFADIAZINE', 'sulfADIAZINE', 'MESTINON', 'ESTINYL', 'ACTIDIL', 'THAM', 'DISIPAL', 'LORFAN', 'EUTONYL', 'BAL', 'NARDIL', 'RAUVAL', 'FLAXEDIL', 'VELBAN', 'EQUANIL', 'DROLBAN', 'PLACIDYL', 'FLAGYL', 'ELAVIL', 'TERFONYL', 'POVAN', 'TEPANIL', 'ESIMIL', 'ISORDIL', 'PLAQUENIL', 'TALWIN', 'ARALEN', 'RITALIN', 'OXSORALEN', 'LIPO GANTRISIN', 'GANTRISIN', 'PBZ', 'AVC', 'DIMETANE', 'ARTANE', 'PAGITANE', 'PAREDRINE', 'PARADIONE', 'PRO-BANTHINE', 'BANTHINE', 'COGENTIN', 'CANTIL', 'BENTYL', 'AVENTYL', 'BENEMID', 'BENYLIN', 'COLBENEMID', 'DOCA', 'URESE', 'TACE', 'CORTISONE ACETATE', 'HYDROCORTISONE ACETATE', 'CORTONE', 'HYDROCORTONE', 'HYDROcodone', 'APRESOLINE', 'PRESAMINE', 'TPN', 'LOCORTEN', 'RIMIFON', 'DELFEN', 'IVADANTIN'